# Init

install and import necessary libraries

In order to run this you must first do two things
1. `pip install llama-cpp-python`
    * (if you get weird errors, consider switching to wsl)

2. install the model
    * `pip3 install huggingface-hub`
    
    * `huggingface-cli download TheBloke/sqlcoder2-GGUF sqlcoder2.Q4_K_M.gguf --local-dir . --local-dir-use-symlinks False`
`


In [5]:
from llama_cpp import Llama
import sqlite3
from sqlite3 import Error

In [6]:
llm = Llama(
      model_path="sqlcoder2.Q4_K_M.gguf",
      n_gpu_layers=32, # Uncomment to use GPU acceleration
      # seed=1337, # Uncomment to set a specific seed
      n_ctx=2048, # Uncomment to increase the context window,
      
)

llama_model_loader: loaded meta data with 19 key-value pairs and 485 tensors from sqlcoder2.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = starcoder
llama_model_loader: - kv   1:                               general.name str              = StarCoder
llama_model_loader: - kv   2:                   starcoder.context_length u32              = 8192
llama_model_loader: - kv   3:                 starcoder.embedding_length u32              = 6144
llama_model_loader: - kv   4:              starcoder.feed_forward_length u32              = 24576
llama_model_loader: - kv   5:                      starcoder.block_count u32              = 40
llama_model_loader: - kv   6:             starcoder.attention.head_count u32              = 48
llama_model_loader: - kv   7:          starcoder.attention.head_count_kv u32              = 1


llama_model_loader: - kv  12:                      tokenizer.ggml.scores arr[f32,49152]   = [0.000000, 0.000000, 0.000000, 0.0000...
llama_model_loader: - kv  13:                  tokenizer.ggml.token_type arr[i32,49152]   = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
llama_model_loader: - kv  14:                      tokenizer.ggml.merges arr[str,48891]   = ["Ġ Ġ", "ĠĠ ĠĠ", "ĠĠĠĠ ĠĠ...
llama_model_loader: - kv  15:                tokenizer.ggml.bos_token_id u32              = 0
llama_model_loader: - kv  16:                tokenizer.ggml.eos_token_id u32              = 0
llama_model_loader: - kv  17:            tokenizer.ggml.unknown_token_id u32              = 0
llama_model_loader: - kv  18:               general.quantization_version u32              = 2
llama_model_loader: - type  f32:  322 tensors
llama_model_loader: - type q4_K:  102 tensors
llama_model_loader: - type q5_K:   40 tensors
llama_model_loader: - type q6_K:   21 tensors
llm_load_vocab: mismatch in special tokens definition

LLAMA-CPP-Python API: https://llama-cpp-python.readthedocs.io/en/latest/api-reference/#high-level-api

In [3]:
llm('I\'m testing you!')['choices'][0]['text']
#if anything prints out that's not an error, it's good


llama_print_timings:        load time =     795.71 ms
llama_print_timings:      sample time =       4.04 ms /    16 runs   (    0.25 ms per token,  3958.44 tokens per second)
llama_print_timings: prompt eval time =     795.63 ms /     6 tokens (  132.60 ms per token,     7.54 tokens per second)
llama_print_timings:        eval time =    3149.28 ms /    15 runs   (  209.95 ms per token,     4.76 tokens per second)
llama_print_timings:       total time =    3987.95 ms /    21 tokens


'\'),\n                         "h\'i there", \'i\\\'ve done a thing to'

In [7]:
prompt = input('What is your question?')

response = llm(f"""## Task
Generate a SQL query to answer the following question:
`{prompt}`

### Database Schema
This query will run on a database whose schema is represented in this string:
CREATE TABLE Bitcoin_History (
  Index INTEGER PRIMARY KEY, -- Unique ID for each product
  Date DATE_FORMAT(), -- Date of the recorded data
  Price DECIMAL(10,2), -- Closing price of Bitcoin on the given date
  Open DECIMAL(10,2), -- Opening price of Bitcoin on the given date
  High DECIMAL(10,2), -- Highest price of Bitcoin on the given date
  Low DECIMAL(10,2), -- Lowest price of Bitcoin on the given date
  Change % DECIMAL(1,10) -- Percentage change in Bitcoin's price from the previous day
  quantity INTEGER  -- Current quantity in stock
);

### SQL
Given the database schema, here is the SQL query that answers `{prompt}`:
```sql""", max_tokens=128)
print(response)


llama_print_timings:        load time =   42797.06 ms
llama_print_timings:      sample time =       8.20 ms /    16 runs   (    0.51 ms per token,  1951.93 tokens per second)
llama_print_timings: prompt eval time =   42796.79 ms /   205 tokens (  208.76 ms per token,     4.79 tokens per second)
llama_print_timings:        eval time =    5924.91 ms /    15 runs   (  394.99 ms per token,     2.53 tokens per second)
llama_print_timings:       total time =   48793.09 ms /   220 tokens


{'id': 'cmpl-fc2b0ac1-0e10-4659-a402-d7fb17836479', 'object': 'text_completion', 'created': 1707603278, 'model': 'sqlcoder2.Q4_K_M.gguf', 'choices': [{'text': '\nSELECT COUNT(DISTINCT date) AS days FROM bitcoin_history;', 'index': 0, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 205, 'completion_tokens': 15, 'total_tokens': 220}}


In [8]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)

    return conn

# def run_task(conn, prompt):
#     """
#     Query all rows in the tasks table
#     :param conn: the Connection object
#     :return:
#     """
#     cur = conn.cursor()
#     cur.execute(prompt)

#     rows = cur.fetchall()

#     for row in rows:
#         print(row)

In [11]:
import pandas as pd
conn = create_connection('sql.db')
response_text = response['choices'][0]['text']
df = pd.read_sql_query(response_text, conn)

In [13]:
df

,days
0,4955


In [14]:
[list(df.columns)] + df.values.tolist()

[['days'], [4955]]

In [ ]:
# conn = create_connection('sql.db')
# response_text = response['choices'][0]['text']
# run_task(conn, response_text)

Warning: You can only execute one statement at a time.